In [1]:
from langchain_core.prompts import ChatPromptTemplate

import os
from dotenv import load_dotenv

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

True

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")

In [3]:
prompt_val = prompt.invoke({"topic": "dog"})
print(prompt_val)

messages=[HumanMessage(content='Tell me an interesting fact about dog', additional_kwargs={}, response_metadata={})]


In [4]:
print(prompt_val.to_messages())

[HumanMessage(content='Tell me an interesting fact about dog', additional_kwargs={}, response_metadata={})]


In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
result = model.invoke(prompt_val)
result

AIMessage(content='Dogs have a remarkable sense of smell, which is estimated to be around 10,000 to 100,000 times more sensitive than that of humans. This extraordinary olfactory capability allows them to detect various scents, including those associated with certain diseases in humans, such as cancer or diabetes, and to track scents over long distances. This is why dogs are often used in search and rescue operations, as well as in medical detection roles.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 14, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C5rhOUlcvcQCyxxN6cSknEM87wYif', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': N

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(result)

'Dogs have a remarkable sense of smell, which is estimated to be around 10,000 to 100,000 times more sensitive than that of humans. This extraordinary olfactory capability allows them to detect various scents, including those associated with certain diseases in humans, such as cancer or diabetes, and to track scents over long distances. This is why dogs are often used in search and rescue operations, as well as in medical detection roles.'

### Now let´s do this LCEL

In [7]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

basicchain = model | output_parser

In [8]:
basicchain.invoke("hello!")

'Hello! How can I assist you today?'

In [9]:
chain = prompt | model | output_parser

chain.invoke({"topic": "dog"})

'One interesting fact about dogs is that they have an extraordinary sense of smell, which is about 10,000 to 100,000 times more acute than that of humans. This incredible olfactory ability allows them to detect scents and pheromones at very low concentrations, making them invaluable in roles such as search and rescue, detecting drugs and explosives, and even identifying certain medical conditions in humans, like low blood sugar or cancer. Their sense of smell is so sophisticated that some dogs can be trained to identify specific scents, making them essential partners in various fields and industries.'

### Retrieval Augmented Generation with LCEL

In [10]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough

embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}
    ),
    Document(
        page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}
    ),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [14]:
retriever.invoke("What does the dog want to eat?")

[Document(id='058e7209-4ffa-4bad-b8da-ae4029ce6071', metadata={'source': 'animal.txt'}, page_content='the dog loves to eat pizza'),
 Document(id='33c05550-1a7b-4e1f-b8e0-8b81928e744a', metadata={'source': 'animal.txt'}, page_content='the cat loves to eat lasagna')]

In [13]:
retriever.invoke("What does the dog want to eat?")

[Document(id='058e7209-4ffa-4bad-b8da-ae4029ce6071', metadata={'source': 'animal.txt'}, page_content='the dog loves to eat pizza'),
 Document(id='33c05550-1a7b-4e1f-b8e0-8b81928e744a', metadata={'source': 'animal.txt'}, page_content='the cat loves to eat lasagna')]

In [15]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4o-mini")

In [16]:
from operator import itemgetter

retrieval_chain = (
    {
        "context": (lambda x: x["question"]) | retriever,
        # "question": lambda x: x["question"],
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [17]:
retrieval_chain.invoke({"question": "What does the dog like to eat?"})

'The dog loves to eat pizza.'

In [19]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4o-mini")

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
retrieval_chain.invoke("What does the dog like to eat?")

'The dog loves to eat pizza.'